In [1]:
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn import model_selection
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
df = pd.read_csv("data.csv",names=["Chemical_formula","A","B","In_literature","Valence_A","Valence_B","Radius_A","Radius_B","Lowest_distortion","Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","a","b","c","alpha","beta","gamma","Vacancy_energy"])

In [3]:
df.drop(df.index[(df["Valence_B"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Vacancy_energy"] =="-")],axis=0,inplace=True)
df['Valence_A'] = df['Valence_A'].replace(['not balanced'], '0') 
df['Valence_A'] = df['Valence_A'].replace(['3'], '1') 
df['Valence_B'] = df['Valence_B'].replace(['not balanced'], '0') 
df['Valence_B'] = df['Valence_B'].replace(['3'], '1') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['cubic'], '0') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['orthorhombic'], '1')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['rhombohedral'], '2')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['tetragonal'], '3')
df['Magnetic_moment'] = df['Magnetic_moment'].replace(['-'], '0.00')
df['Vacancy_energy'] = df['Vacancy_energy'].replace(['-'], '0.00')
df['In_literature'] = df['In_literature'].replace(["FALSE"], '0') 
df['In_literature'] = df['In_literature'].replace(["TRUE"], '1')

In [4]:
df["In_literature"]

0       In_literature
75                  0
76                  0
79                  0
80                  0
            ...      
5322                0
5326                0
5327                0
5328                0
5329                0
Name: In_literature, Length: 2640, dtype: object

In [5]:
df[1:]

,Chemical_formula,A,B,In_literature,Valence_A,Valence_B,Radius_A,Radius_B,Lowest_distortion,Formation_energy,...,Magnetic_moment,Volume_per_atom,Band_gap,a,b,c,alpha,beta,gamma,Vacancy_energy
75,Ag2O3,Ag,Ag,0,0,0,1.28,0.95,0,0.351,...,0.00,12.997,0,4.02,4.02,4.02,90,90,90,-2.694
76,AgAlO3,Ag,Al,0,0,0,1.28,0.54,1,-1.51,...,0.00,9.898,0,5.094,5.323,7.3,90,90,90,-1.341
79,AgBO3,Ag,B,0,1,1,0.75,0.27,0,-0.35,...,0.00,8.138,0,3.439,3.439,3.439,90,90,90,-2.641
80,AgBaO3,Ag,Ba,0,0,0,1.28,1.35,1,-0.989,...,0.00,15.631,0.247,6.636,6.739,6.99,90,90,90,-6.975
81,AgBeO3,Ag,Be,0,0,0,1.28,0.45,0,-0.616,...,0.00,8.781,0,3.528,3.528,3.528,90,90,90,-3.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322,ZrTmO3,Zr,Tm,0,0,0,0.89,0.96,0,-2.315,...,0.2,14.496,0,4.169,4.169,4.169,90,90,90,-3.645
5326,ZrYO3,Zr,Y,0,0,0,0.89,0.9,0,-2.126,...,0.2,15.277,0,4.243,4.243,4.243,90,90,90,-4.92
5327,ZrYbO3,Zr,Yb,0,0,0,0.89,0.95,1,-3.455,...,0,13.136,4.007,5.558,5.726,8.254,90,90,90,-6.177
5328,ZrZnO3,Zr,Zn,0,0,0,0.89,0.74,0,-1.63,...,0.001,10.804,0,3.78,3.78,3.78,90,90,90,-0.762


In [23]:
features=["Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","Vacancy_energy"]
#,"Radius_A","Radius_B","Valence_A","Valence_B"
X = df.loc[1:, features].values
Y = df.loc[1:, "Lowest_distortion"].values

In [24]:
X

array([['0.351', '0.576', '0.00', ..., '0.95', '0', '0'],
       ['-1.51', '0.235', '0.00', ..., '0.54', '0', '0'],
       ['-0.35', '1.074', '0.00', ..., '0.27', '1', '1'],
       ...,
       ['-3.455', '0.205', '0', ..., '0.95', '0', '0'],
       ['-1.63', '1.21', '0.001', ..., '0.74', '0', '0'],
       ['-2.382', '0.906', '0.382', ..., '0.72', '0', '0']], dtype=object)

In [25]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.2, random_state=25)

In [26]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=1, n=678 (32.117%)
Class=2, n=101 (4.784%)
Class=0, n=1293 (61.251%)
Class=3, n=39 (1.847%)


In [27]:
oversample = SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [28]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=1, n=1293 (25.000%)
Class=2, n=1293 (25.000%)
Class=0, n=1293 (25.000%)
Class=3, n=1293 (25.000%)


In [29]:
#without weights
SVM_linear = svm.SVC(kernel='linear', C=1).fit(X_train, Y_train)
print(SVM_linear.score(X_test, Y_test))

0.8768939393939394


In [30]:
#setting weights so that points which exsist in literature has more weight
weights=[]
for i in df["In_literature"]:
    if i=='0':
        weights=np.append(weights,np.array([1]))
    if i=='1':
        weights=np.append(weights,np.array([5]))
    else:
        pass
weights.shape

(2639,)

In [14]:
#with weights
SVM_linear2 = svm.SVC(C=50,gamma=1)
print(X_train.shape)
print(Y_train.shape)
SVM_linear2.fit(X_train, Y_train, sample_weight=weights)
print(SVM_linear2.score(X_test,Y_test))

(5172, 7)
(5172,)


ValueError: sample_weight and X have incompatible shapes: (2639,) vs (5172, 7)
Note: Sparse matrices cannot be indexed w/boolean masks (use `indices=True` in CV).

In [ ]:
#with weights
